<h1 style="text-align: center;" markdown="1">Complex Networks - Practical Session 3</h1>
<h3 style="text-align: center;" markdown="1">by Dimitri Lajou and Fabrice Lebeau</h3>

In [ ]:
import numpy as np
import math
import random
import scipy
import scipy.special
import matplotlib
import matplotlib.pyplot as plt;
%matplotlib inline  
plt.rcParams['figure.figsize'] = (15, 9)
plt.rcParams['font.size'] = 14
from IPython.display import Math, Markdown, Latex, display, display_latex, SVG
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [ ]:
""" CR15 graph library """
class Graph(object):

    def __init__(self, graph_dict={}, graph_name=""):
        """ initializes a graph object """
        self.__graph_dict = graph_dict.copy()
        self.__name=graph_name

    def name(self):
        return self.__name
        
    def vertices(self):
        """ returns the vertices of a graph """
        return list(self.__graph_dict.keys())

    def edges(self):
        """ returns the edges of a graph """
        return self.__generate_edges()
    
    def neighbours(self, u):
        return self.__graph_dict[u]
    
    def connected_components(self):
        return self.__generate_components()

    def add_vertex(self, vertex):
        """ If vertex is not in self.__graph_dict, a key "vertex" with an empty
        list as a value is added to the dictionary. Otherwise nothing has to be 
        done."""
        if not vertex in self.__graph_dict:
            self.__graph_dict[vertex] = []
        

    def add_edge(self, edge):
        """ assumes that edge is of type set, tuple or list. No loops or 
        multiple edges."""
        my_edge = list(edge)
        if len(my_edge) != 2: return
        u = edge.pop()
        v = edge.pop()
        if u in self.__graph_dict and v in self.__graph_dict:
            if u != v:
                if v not in self.__graph_dict[u]:
                    self.__graph_dict[u].append(v)
                if u not in self.__graph_dict[v]:
                    self.__graph_dict[v].append(u)
    
    def remove_edge(self, edge):
        my_edge = list(edge)
        if len(my_edge) != 2: raise WrongSizeForEdge()
        u = edge.pop()
        v = edge.pop()
        if u in self.__graph_dict and v in self.__graph_dict:
            self.__graph_dict[v].remove(u)
            self.__graph_dict[u].remove(v)

    def __generate_edges(self):
        """ A static method generating the edges of the graph "graph". Edges 
        are represented as sets two vertices, with no loops. To complete."""
        edges = []
        for v, edges_list in self.__graph_dict.items():
            for u in edges_list:
                if v < u:
                    edges.append(set([v,u]))
        return edges
    
    def rewire(self, u, v):
        if v not in self.__graph_dict[u]:
            return
        not_connected_to = [w for w in self.vertices() if w not in self.__graph_dict[u]]
        if len(not_connected_to) != 0:
            w = not_connected_to[random.randrange(len(not_connected_to))]
            self.remove_edge({u,v})
            self.add_edge({u,w})
    
    def vertex_degree(self, vertex):
        """Return a dictionary degree"""
        return len(self.__graph_dict[vertex])
    
    def vertex_degrees(self):
        """Return a dictionary degree"""
        degrees = {}
        for v, edges_list in self.__graph_dict.items():
            degrees[v] = len(edges_list)
        return degrees
    
    def vertex_degree_simple(self, vertex):
        """Return a dictionary degree"""
        edges = set(self.__graph_dict[vertex])
        if vertex in edges:
            return len(edges) - 1
        else:
            return len(edges)    
            
    def vertex_degrees_simple(self):
        """Return a dictionary degree"""
        degrees = {}
        for v in self.__graph_dict:
            degrees[v] = self.vertex_degree_simple(v)
        return degrees
    
    def degree_distrib_simple(self):
        distrib = {}
        for v in self.__graph_dict:
            k = self.vertex_degree_simple(v)
            if k in distrib:
                distrib[k] += 1
            else:
                distrib[k] = 1
        for k in distrib:
            distrib[k] /= len(self.__graph_dict)
        return distrib
    
    
    def find_isolated_vertices(self):
        """Return a set of zero-degree verticies"""
        zero_set = set()
        for v, edges_list in self.__graph_dict.items():
            if len(edges_list) == 0:
                zero_set.add(v)
        return zero_set
    
    def has_isolated_vertices(self):
        """Return a set of zero-degree verticies"""
        for v, edges_list in self.__graph_dict.items():
            if len(edges_list) == 0:
                return True
        return False
                
    def density(self):
        """Return the density of the graph"""
        deg = self.vertex_degrees()
        density = 0
        for v, d in deg.items():
            density += d
        density /=  (len(deg) -1) *len(deg)
        return density
                    
    def dict(self):
        return self.__graph_dict
    
    def degree_sequence(self):
        """Return the list of vertex degree sorted by decreasing degree"""
        deg = self.vertex_degrees()
        deg_list = [v for v in deg.values()]
        deg_list.sort(reverse=True)
        return tuple(deg_list)
    
    @staticmethod
    def erdos_gallai(deg_seq):
        """Given a degree sequence, this method verify that this sequence verify the erdos gallai conditions"""
        even_number = 0
        for v in deg_seq:
            even_number += v
        if even_number % 2 == 1 :
            return False
        sumOfdi = 0
        for k in range(len(deg_seq)):
            sumOfdi += deg_seq[k]
            sumOfMin = k*(k+1)
            for i in range(k, len(deg_seq)):
                sumOfMin += min(deg_seq[i], k+1)
            if sumOfMin < sumOfdi:
                return False
        return True
    
    def global_clustering_coefficient(self):
        triangle = 0
        triplet = 0
        for v in self.__graph_dict:
            for u in self.__graph_dict[v]:
                for w in self.__graph_dict[v]:
                    if u != w: 
                        triplet += 1
                    if u != w and w in self.__graph_dict[u]:
                        triangle += 1
        return triangle / triplet
    
    """ Graph traversal """
    def components_BFS(self, vertices, comps, base_vertice):
        if not vertices:
            return
        new_vertices = []
        for v in vertices:
            comps[v] = base_vertice
            for u in self.__graph_dict[v]:
                if comps[u] == u and u != base_vertice:
                    new_vertices.append(u)
        self.components_BFS(new_vertices, comps, base_vertice)
    
    def __generate_components(self):
        """Retuen a dictionnary of components representant"""
        comps = {}
        for u in self.__graph_dict:
            comps[u] = u
        for u in self.__graph_dict:
            if comps[u] == u:
                self.components_BFS([u], comps, u)
        return comps
    
    def shortest_path_BFS(self, vertices, seen, d, goal):
        if not vertices:
            return math.inf
        if goal in vertices:
            return d
        new_vertices = set()
        for v in vertices:
            seen.add(v)
        for v in vertices:
            for u in self.__graph_dict[v]:
                if u not in seen:
                    new_vertices.add(u)
        return self.shortest_path_BFS(new_vertices, seen, d+1, goal)
    
    def shortest_path(self, s, t):
        """Return the shortest path between s and t"""
        return self.shortest_path_BFS({s}, set(), 0, t)
    
    def diameter_BFS(self, vertices, seen, d):
        if not vertices:
            if d > 0:
                return d-1
            else:
                return d
        new_vertices = set()
        for v in vertices:
            seen.add(v)
        for v in vertices:
            for u in self.__graph_dict[v]:
                if u not in seen:
                    new_vertices.add(u)
        return self.diameter_BFS(new_vertices, seen, d+1)
    
    def diameter(self):
        diam = 0
        for u in self.__graph_dict:
            m = self.diameter_BFS({u}, set(), 0)
            if (m > diam):
                diam = m
        return diam
    
    def total_length_shortest_paths_BFS(self, vertices, seen, d):
        if not vertices:
            return 0
        new_vertices = set()
        increment = 0
        for v in vertices:
            if v not in seen:
                increment += d
            seen.add(v)
        for v in vertices:
            for u in self.__graph_dict[v]:
                if u not in seen:
                    new_vertices.add(u)
        return increment + self.total_length_shortest_paths_BFS(new_vertices, seen, d+1)
    
    def average_shortest_paths_length(self):
        n = len(self.vertices())
        if n <= 1:
            return 0
        average = 0
        for u in self.connected_components().values():
            average += self.total_length_shortest_paths_BFS({u}, set(), 0)
        return average / (n * (n-1.))
    
    def diameter_component(self, u):
        """ Return the diameter of the component containing vertex u """
        component = set()
        comps = self.connected_components()
        # First get the nodes of the component #
        for v in self.__graph_dict:
            if comps[v] == comps[u]:
                component.add(v)
        diam = 0
        for v in component:
            m = self.diameter_BFS({v}, set(), 0)
            if (m > diam):
                diam = m
        return diam
    
    def biggest_component_diameter(self):
        if not self.vertices(): return 0
        
        # First determine the biggest component #
        comps = self.connected_components()
        comps_size = {}
        for u in comps.values():
            comps_size[u] = 0
        for v,u in comps.items():
            comps_size[u] += 1
        biggest = tuple(comps.values())[0]
        max_size = comps_size[biggest]
        for u in comps.values():
            if comps_size[u] > max_size:
                max_size = comps_size[u]
                biggest = u
        
        # Return the diameter of the corresponding component
        return self.diameter_component(biggest)
        
    def spanning_tree(self):
        queue = []
        tree = []
        # Hack to get one key in the dict
        for v in self.__graph_dict:
            queue.append(v)
            break
        seen = [queue[-1]]

        while queue:
            u = queue.pop()
            for v in self.__graph_dict[u]:
                if not v in seen:
                    tree.append(set([u,v]))
                    seen.append(v)
                    queue.append(v)
        return tree
    
    def irregular_edge_count(self):
        nb_loop = 0
        nb_multi = 0
        nb_edge = 0
        for v in self.__graph_dict:
            seen = []
            for u in self.__graph_dict[v]:
                nb_edge += 1
                if u == v:
                    nb_loop += 1
                elif u in seen:
                    nb_multi += 1
                else:
                    seen.append(u)
        return (nb_loop + nb_multi // 2) / nb_edge * 2
        
    """ Static methods for defining classical graphs """
    @staticmethod
    def clique(n):
        d = {}
        s = set(i+1 for i in range(n))
        for i in range(n):
            d[i+1] = s.difference(set({i+1}))
        return Graph(d,"$K_"+str(n)+"$")
    
    @staticmethod
    def no_edges(n):
        d = {}
        for i in range(n):
            d[i+1] = []
        return Graph(d,"$D_"+str(n)+"$")
    
    @staticmethod
    def er_np(n, p):
        graph_dict = {}
        for i in range(n):
            graph_dict[i] = []
        for i in range(n):
            for j in range(i):
                rv = random.uniform(0,1)
                if rv < p:
                    graph_dict[i].append(j)
                    graph_dict[j].append(i)  
        return Graph(graph_dict)

    def er_nm(n, m):
        graph_dict = {}
        for i in range(n):
            graph_dict[i] = []
        # number of possible edges (*2 since it is easier to sample couple than pairs)
        nb_possibility = n*n
        count_left = m
        if m > n*(n-1)/2 :
            count_left = n*(n-1)/2
        while count_left > 0:
            # sample an index for the new edge
            edge_ind = random.randrange(nb_possibility)
            # get the two endpoints (part where it is easier for couple)
            i = edge_ind // n
            j = edge_ind % n
            # handle loops and multiedges
            if i == j :
                continue 
            if j in graph_dict[i]:
                continue
            graph_dict[i].append(j)
            graph_dict[j].append(i)
            count_left -= 1
        return Graph(graph_dict)

    """ Importing from a text file """
    @staticmethod
    def from_txt(file):
        G = Graph()
        lines = open(file).readlines()
        for l in lines:
            p = parse('{:d}\t{:d}', l)
            G.add_vertex(p[0])
            G.add_vertex(p[1])
            G.add_edge({p[0],p[1]})
        return G
    
    def print(self):
        print(self.__graph_dict)

## The Watts-Strogatz model
Added `rewire` method in `Graph` class.

In [ ]:
def regular_ring_lattice(N, k):
    G = Graph()
    for i in range(N):
        G.add_vertex(i)
    for i in range(N):
        for j in range(N):
            if i >= j:
                continue
            if 0 < math.fabs(i-j) % (N-1-k/2) <= k/2:
                G.add_edge({i,j})
    return G
def watts_strogatz(N, k, beta):
    G = regular_ring_lattice(N, k)
    for u in range(N):
        for v in G.neighbours(u):
            if random.uniform(0., 1.) < beta:
                G.rewire(u, v)
    return G

In [ ]:
# Comparing global clustering coefficient of WS and ER models
nb_points = 100
n_min = 10
n_max = 120
nb_average = 10

def compare_clustering_coeff(beta):
    step = (n_max - n_min) // nb_points
    if step == 0:
        step = 1
    x = [n_min + step * i for i in range(nb_points)]
    coeffs_ws = [0 for i in range(nb_points)]
    coeffs_er = [0 for i in range(nb_points)]
    for t in range(nb_average):
        for i in range(nb_points):
            N = n_min + step * i
            k = 1 + (N//10)
            coeffs_ws[i] += watts_strogatz(N, k, beta).global_clustering_coefficient()
            coeffs_er[i] += Graph.er_nm(N, N * k // 2).global_clustering_coefficient()
    coeffs_ws = [y / nb_average for y in coeffs_ws]
    coeffs_er = [y / nb_average for y in coeffs_er]
    
    plt.plot(x, coeffs_ws, label='Watts-Strogatz')
    plt.plot(x, coeffs_er, label='Erdös-Rényi')
    plt.xlabel('$N$')
    plt.ylabel('Global clustering coeficient')
    
    plt.legend()
    plt.show()
    
w = interact(compare_clustering_coeff
            ,beta = widgets.FloatSlider(description='$\\beta$',min=0., max=1.,value=0.8,step=0.01,continuous_update=False))

In [ ]:
# Comparing average shortest path lengths of WS and ER models
nb_points = 100
n_min = 10
n_max = 120
nb_average = 10

def compare_average_shortest_path_lengths(beta):
    step = (n_max - n_min) // nb_points
    if step == 0:
        step = 1
    x = [n_min + step * i for i in range(nb_points)]
    coeffs_ws = [0 for i in range(nb_points)]
    coeffs_er = [0 for i in range(nb_points)]
    for t in range(nb_average):
        for i in range(nb_points):
            N = n_min + step * i
            k = 1 + (N//10)
            coeffs_ws[i] += watts_strogatz(N, k, beta).global_clustering_coefficient()
            coeffs_er[i] += Graph.er_nm(N, N * k // 2).global_clustering_coefficient()
    coeffs_ws = [y / nb_average for y in coeffs_ws]
    coeffs_er = [y / nb_average for y in coeffs_er]
    
    plt.plot(x, coeffs_ws, label='Watts-Strogatz')
    plt.plot(x, coeffs_er, label='Erdös-Rényi')
    plt.xlabel('$N$')
    plt.ylabel('Average shortest paths length')
    
    plt.legend()
    plt.show()
    
w = interact(compare_average_shortest_path_lengths
            ,beta = widgets.FloatSlider(description='$\\beta$',min=0., max=1.,value=0.1,step=0.01,continuous_update=False))

In [ ]:
# Influence of beta on clustering coeff and average path lengths
nb_points = 100
nb_average = 10

def beta_influence(N, k):
    beta_min = 0.05
    beta_max = 1.
    step = (beta_max - beta_min) / nb_points
    x = [beta_min + step * i for i in range(nb_points)]
    clustering = [0 for i in range(nb_points)]
    aspl = [0 for i in range(nb_points)]
    for t in range(nb_average):
        for i in range(nb_points):
            beta = beta_min + i * step
            G = watts_strogatz(N, k, beta)
            clustering[i] += G.global_clustering_coefficient()
            aspl[i] += G.average_shortest_paths_length()
            
    clustering = [y / nb_average for y in clustering]
    aspl = [y / nb_average for y in aspl]
    
    plt.plot(x, clustering, label='Clustering coefficient')
    plt.plot(x, aspl, label='Average shortest path length')
    plt.xlabel('$\\beta$')
    
    plt.legend()
    plt.show()
    
w = interact(beta_influence
            ,N = widgets.IntSlider(description='$N$',min=10,max=150,value=100,step=5,continuous_update=False)
            ,k = widgets.IntSlider(description='$k$',min=10,max=150,value=10,step=5,continuous_update=False))